# Atelier Keakr

Travail sur un système de recommandation de contenu.

Ici, on se connecte à la base.

In [116]:
from datetime import datetime, timedelta
from pyArango.connection import *
conn = Connection(arangoURL="https://db-datacraft.staging.keakr.io/",
username="root", password="vDWSyr2fPfkaZkUw")

Ici, on choisit la BDD keakr

In [3]:
db = conn["keakr"]

***

***

Données des utilisateurs
---

Cette fonction permet d'extraire les informations relatives à un ou plusieurs utilisateurs keakr, sur la base de leur individuel.  
**Remarque :** Il faut filtrer cette requête sur des utilisateurs donnés pour ne pas surcharger la base de données (par exemple, ceux récupérés via une autre requête, comme les utilisateurs les plus actifs).

![exemple de sortie](./utils/Keakr_fct_user.png)

In [52]:
def get_user_info(user_id):
    aql = "for u in users filter u._id in {}".format(user_id)+" return {id: u._key, createdAt: u.createdAt,lastConnectionDate: u.lastConnectionDate,usedGenres: u.profile.genres,preferredGenres: (for p in attributes(u.preferences) for g in attributes(u.preferences[p].genres) collect genreId = g aggregate count = sum(u.preferences[p].genres[g]) sort count desc return {genreId, count}), stats: u.stats, isBeatmaker: u.stats.beatCount > 0, isSinger: u.stats.keakCount > 0}"
    return db.AQLQuery(aql, rawResults=True)


In [200]:
users = ["users/17301813629699063750", "users/17301813627868798088"] #, "users/17301813625254286371", "users/6132767409", "users/17301813629883872232", "users/17301813628105104258", "users/65324880", "users/17301813625444199021", "users/17301813629478571151", "users/63969001", "users/17301813622852507908", "users/17301813621614964195"]

query = get_user_info(["users/17301813629699063750", "users/17301813627868798088"])
print(len(query))
print(query[:1])

2
[{'id': '17301813627868798088', 'createdAt': '2021-04-27T14:29:55.7851529Z', 'lastConnectionDate': '2022-02-09T09:34:27.1083572Z', 'usedGenres': [{'genreId': '12468689066', 'count': 2}, {'genreId': '17301813621692872888', 'count': 31}, {'genreId': '17301813621692872889', 'count': 3}, {'genreId': '17301813621692872891', 'count': 3}, {'genreId': '17301813621692872893', 'count': 1}, {'genreId': '17301813621692872894', 'count': 9}, {'genreId': '17301813621692872896', 'count': 2}, {'genreId': '17301813623043473093', 'count': 60}, {'genreId': '9920897514', 'count': 7}, {'genreId': '9920897516', 'count': 3}, {'genreId': '9920897517', 'count': 16}, {'genreId': '9920897519', 'count': 17}, {'genreId': '9920897520', 'count': 2}, {'genreId': '9920897521', 'count': 2}, {'genreId': '9920897522', 'count': 5}, {'genreId': '9920897523', 'count': 4}, {'genreId': '9920897524', 'count': 2}, {'genreId': '9920897525', 'count': 8}, {'genreId': '9920897526', 'count': 2}, {'genreId': '9920897527', 'count': 2

----

Données d'audience
---

Cette fonction permet d'obtenir des relevés d'audience : une ligne représente "tel utilisateur a vu tel contenu tel jour et il en a regardé 0%, 25%, 50%, 75% ou 100%"

![exemple de sortie](./utils/Keakr_fct_audience.png)

In [191]:
def get_audience_info(start_period=None, end_period=None, limit=100):
    if end_period is None :
        end_period = datetime.today().strftime('%Y-%m-%d')
    if start_period is None :
        start_period = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    aql = "for v in views filter v.timestamp >= '{}' filter v.timestamp <= '{}'".format(start_period, end_period)+" filter PARSE_IDENTIFIER(v.contentId).collection == 'keaks' return {userId: v.userId, contentId: v.contentId, timestamp: v.timestamp, progress: v.progress}"
    return db.AQLQuery(aql, rawResults=True, batchSize=limit)

In [195]:
query = get_audience_info(start_period='2022-02-01', end_period='2022-02-15', limit=100000)

print(len(query))

85834


----

Données d'engagement sur le contenu
---

Cette fonction permet d'obtenir des indicateurs relatifs à l'engagement sur les contenu de la part des utilisateurs : nombres de vus, commentaires, 'likes', pourcentage moyen de visionage et des informations sur l'instrumentale.

![exemple de sortie](./utils/Keakr_fct_engagement.png)

In [159]:
def get_engagement_info(limit=250):
    aql = """for k in keaks 
    let basedOn = first(for i in isBasedOn filter i._from == k._id limit 1 return i._to)
    filter basedOn
    let beat = document(basedOn)
    let genres = (for g in beat.genres let genre = document(beatGenres, g) return {genreId: genre._key, name: genre.name})"""
    
    aql += " limit {}".format(limit)
    
    aql += " return {id: k._id, createdAt: k.createdAt, media: k.hlsStreamingUrl, likeCount: k.stats.likeCount, commentCount: k.stats.commentCount, viewCount: k.stats.viewCount, averageProgress: k.averageViewProgress ? k.averageViewProgress : 0, genres: genres}"

    return db.AQLQuery(aql, rawResults=True, batchSize=limit)


In [201]:
query = get_engagement_info(limit=100)
print(len(query))
print(query[0])

100
{'id': 'keaks/17301813623064450175', 'createdAt': '2019-10-10T21:20:59.2970322Z', 'media': 'https://cdn.keakr.com/media/hls/6678b82d-c7c1-4958-9505-247160a7aeac/master.m3u8', 'likeCount': 4, 'commentCount': 2, 'viewCount': 42, 'averageProgress': '7.80', 'genres': [{'genreId': '9920897543', 'name': 'Trap'}]}


----

Données du graphe social
---

Cette fonction permet d'obtenir des indicateurs relatifs à l'engagement sur les contenu de la part des utilisateurs : nombres de vus, commentaires, 'likes', pourcentage moyen de visionage et des informations sur l'instrumentale.

![exemple de sortie](./utils/Keakr_fct_social_graph.png)

In [170]:
def get_social_graph_info(users, limit):
    aql = """for f in follows 
             filter f._from in {}
             limit {}""".format(users, limit)
    aql += " return {from: f._from, to: f._to, since: f.createdAt, mutual: f.isMutual}"
    
    return db.AQLQuery(aql, rawResults=True, batchSize=limit)


In [175]:
users = ["users/17301813629699063750", "users/17301813627868798088", "users/17301813625254286371", "users/6132767409", "users/17301813629883872232", "users/17301813628105104258",
 "users/65324880", "users/17301813625444199021", "users/17301813629478571151", "users/63969001", "users/17301813622852507908", "users/17301813621614964195"]

query = get_social_graph_info(users=users, limit=20)
print(len(query), len(users))

20 12


---

Vidéos les plus vues
---

Cette fonction extrait les n vidéos qui comptabilisent le plus grand nombre de vues sur une période de temps donnée.  
Si aucun argument ne lui est donnée, elle extrait les 100 vidéos les plus vues sur les trente derniers jours par rapport au moment d'exécution.

![exemple de sortie](./utils/Keakr_fct_most_watched_video.png)

In [188]:
def get_most_watched_video(start_period=None, end_period=None, limit=100):
    if end_period is None :
        end_period = datetime.today().strftime('%Y-%m-%d')
    if start_period is None :
        start_period = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    aql = "for v in views filter v.timestamp >= '{}' filter v.timestamp <= '{}'".format(start_period, end_period)
    aql += """collect contentId = v.contentId with count into count
            sort count desc
            limit {}""".format(limit)
    aql += "return {contentId, count}"

    return db.AQLQuery(aql, rawResults=True, batchSize=limit)

In [ ]:
query = get_most_watched_video(start_period='2021-01-01', end_period='2022-03-22', limit=10)

print(len(query))
# print(query[:3])